####  Этап 1: Оценка проблемы — Сводный анализ отключений

#### На этом этапе мы анализируем таблицу `blackouts`, чтобы понять **масштаб, длительность и характер** проблемы отключения коммунальных услуг в целом.
 
#### **Цель:**
##### 1.  Оценить общее количество и продолжительность отключений.
##### 2.  Выявить самые проблемные типы услуг (электричество, вода, отопление).
##### 3.  Определить основные организации-инициаторы отключений.

In [1]:
# 1.1. Настройка окружения и загрузка данных

# Используем библиотеку `sqlite3` для подключения к базе данных и `pandas` для анализа и обработки данных.

import sqlite3
import pandas as pd
import numpy as np

# Установка опций отображения
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Подключение к базе данных
DB_PATH = '../databases/dataset.db' 
conn = sqlite3.connect(DB_PATH)

# Загрузка таблицы blackouts
query_blackouts = "SELECT * FROM blackouts;"
blackouts_df = pd.read_sql_query(query_blackouts, conn)

print(f"Загружено записей в blackouts: {len(blackouts_df)}")
blackouts_df.head()

Загружено записей в blackouts: 25264


,id,start_date,end_date,description,type,initiator_name,source
0,f88cefa506f44ebf8f010b8681b5449e,2018-01-01 00:08:00,2018-01-01 09:00:00,"Авария на сети электроснабжения, ведутся восст...",electricity,МУПВ ВПЭС (электрические сети Ленинского района),Единая дежурная диспетчерская служба города (Л...
1,38ddf6852801fa90cc70f9770239961e,2018-01-01 00:24:00,2018-01-01 10:00:00,"Авария на электролинии, остановка работы насос...",cold_water,МУПВ ВПЭС (электрические сети Ленинского района),Единая дежурная диспетчерская служба города (Л...
2,53c570099fe380dce9e56e2ace9cfa9c,2018-01-01 10:44:00,2018-01-02 18:00:00,Авария в системе водоснабжения дома. Жителям н...,hot_water,"ООО ""Управляющая компания ""Регион-ЖКХ""","Аварийная служба ООО ""Мадикс"""
3,9c1b9ebbd9a698eef046b27cb3568745,2018-01-01 11:32:00,2018-01-01 15:00:00,"Авария на сети электроснабжения, ведутся восст...",electricity,МУПВ ВПЭС (электрические сети Фрунзенского рай...,Единая дежурная диспетчерская служба города (Ф...
4,8aa631cb343aac0731bbde806dfa6d8c,2018-01-01 11:33:00,2018-01-01 15:00:00,"Авария на электролинии, остановка работы насос...",hot_water,МУПВ ВПЭС (электрические сети Фрунзенского рай...,Единая дежурная диспетчерская служба города (Ф...


In [2]:
# 1.2. Подготовка данных: Расчет длительности

# Конвертируем столбцы с датами в формат datetime и рассчитываем продолжительность каждого отключения в часах.

# Преобразование столбцов с датами в формат datetime
blackouts_df['start_date'] = pd.to_datetime(blackouts_df['start_date'])
blackouts_df['end_date'] = pd.to_datetime(blackouts_df['end_date'])

# Расчет длительности отключения (timedelta)
blackouts_df['duration'] = blackouts_df['end_date'] - blackouts_df['start_date']

# Расчет длительности в часах (для удобства анализа)
blackouts_df['duration_hours'] = blackouts_df['duration'].dt.total_seconds() / 3600

# Удаление записей с отрицательной или нулевой длительностью (ошибочные данные или мгновенные события)
blackouts_df = blackouts_df[blackouts_df['duration_hours'] > 0]

print(f"Количество записей после очистки: {len(blackouts_df)}")
blackouts_df[['start_date', 'end_date', 'duration', 'duration_hours']].describe()

Количество записей после очистки: 25264


,start_date,end_date,duration,duration_hours
count,25264,25264,25264,25264.000000
mean,2018-12-29 18:52:18.816497408,2018-12-31 02:06:20.649936640,1 days 07:14:01.833438885,31.233843
min,2018-01-01 00:08:00,2018-01-01 09:00:00,0 days 00:01:00,0.016667
25%,2018-07-08 09:36:00,2018-07-12 12:00:00,0 days 02:52:00,2.866667
50%,2018-12-27 16:36:00,2018-12-28 08:14:30,0 days 04:41:00,4.683333
75%,2019-06-18 11:14:00,2019-06-19 16:00:00,0 days 10:45:00,10.750000
max,2019-12-31 20:29:00,2020-01-09 12:00:00,355 days 23:13:00,8543.216667
std,NaN,NaN,6 days 16:21:31.850660568,160.358847


In [3]:
#  1.3. Сводный анализ длительности и частоты

# Рассчитаем ключевые метрики, чтобы оценить общий масштаб проблемы для пользователей.


# Общее количество отключений
total_blackouts = len(blackouts_df)

# Общая длительность отключений (в часах и днях)
total_duration_hours = blackouts_df['duration_hours'].sum()
total_duration_days = total_duration_hours / 24

# Статистика по длительности одного отключения
avg_duration = blackouts_df['duration_hours'].mean()
median_duration = blackouts_df['duration_hours'].median()
max_duration = blackouts_df['duration_hours'].max()

print("--- Общие метрики отключений ---")
print(f"Всего отключений: {total_blackouts:,}")
print(f"Общее время, проведенное без услуг (суммарно по всем отключениям):")
print(f"  {total_duration_hours:,.2f} часов, или {total_duration_days:,.2f} дней")
print("-" * 35)
print("Статистика длительности одного отключения (часы):")
print(f"  Средняя длительность: {avg_duration:.2f} ч.")
print(f"  Медианная длительность: {median_duration:.2f} ч. (более устойчивый показатель)")
print(f"  Максимальная длительность: {max_duration:.2f} ч.")

--- Общие метрики отключений ---
Всего отключений: 25,264
Общее время, проведенное без услуг (суммарно по всем отключениям):
  789,091.80 часов, или 32,878.83 дней
-----------------------------------
Статистика длительности одного отключения (часы):
  Средняя длительность: 31.23 ч.
  Медианная длительность: 4.68 ч. (более устойчивый показатель)
  Максимальная длительность: 8543.22 ч.


 **Вывод по сводному анализу:**

 Проблема отключений характеризуется высокой частотой (более 25 тысяч событий) и ощутимой продолжительностью (медиана 4.68 часа). Такое сочетание делает простое информирование о факте отключения недостаточным.

Ценность фичи предсказания заключается в следующем: зная, что типичное отключение длится почти 5 часов, пользователи, получившие прогноз, смогут заранее принять меры (например, перенести бытовые дела, изменить свое расписание дня, или уехать на время отключения). Это превращает неопределенность и вынужденный простой в управляемое ожидание, что напрямую повышает удовлетворенность пользователей.


In [4]:

# ## 1.4. Анализ по типу услуг (`type`)
# Распределение отключений по типам услуг (`electricity`, `water`, `heating`) для выявления наиболее критичных областей.

type_analysis = blackouts_df.groupby('type').agg(
    count=('id', 'count'),
    total_duration_hours=('duration_hours', 'sum'),
    average_duration_hours=('duration_hours', 'mean'),
    median_duration_hours=('duration_hours', 'median')
).sort_values(by='count', ascending=False)

# Добавление столбца с долей от общего числа отключений
type_analysis['count_share'] = (type_analysis['count'] / total_blackouts) * 100

print("--- Анализ отключений по типу услуги ---")
print(type_analysis.round(2))

--- Анализ отключений по типу услуги ---
             count  total_duration_hours  average_duration_hours  median_duration_hours  count_share
type                                                                                                
hot_water     9339             650733.10                   69.68                   6.00        36.97
cold_water    9109              93630.82                   10.28                   4.65        36.06
electricity   4745              21933.92                    4.62                   3.20        18.78
heat          2071              22793.97                   11.01                   5.25         8.20



Вывод по Анализу Отключений по Типу Услуг

Ключевые наблюдения и их значение:
1) Горячая вода (hot_water)	Доля в общем числе: 36.97%,	Медианная длительность: 6.00 ч.
Самый частый тип отключений и самый продолжительный по медиане. Предсказание критически важно, так как затрагивает наибольшее число событий и приносит наибольший дискомфорт (долгие отключения).

2) Холодная вода (cold_water)	Доля в общем числе: 36.06%, Медианная длительность: 4.65 ч.
Почти столь же часто, как горячая вода. Продолжительность также высока. Ключевой приоритет для предсказания, поскольку отсутствие холодной воды является серьезной бытовой проблемой.

3) Электричество (electricity)	Доля в общем числе: 18.78%, Медианная длительность: 3.20 ч.
Менее часто, но всё еще значительно. Имеет самую низкую медианную длительность. Тем не менее, критично для работы и жизнеобеспечения, поэтому должно быть включено в предсказание.

4) Отопление (heat)	Доля в общем числе: 8.20%, Медианная длительность: 5.25 ч.
Наименее частый, но имеет вторую по продолжительности медиану. Отключение отопления в холодное время года может быть самым критическим с точки зрения последствий.

Обоснование приоритетов фичи:

1) Приоритет №1: Водоснабжение (Горячая и Холодная вода): Составляют более 73% всех отключений. Это является основным фокусом для фичи предсказания, так как пользователи чаще всего сталкиваются именно с этой проблемой, и она длится долго (4.65–6.00 часов).

2) Приоритет №2: Отопление: Несмотря на низкую частоту, высокая медианная длительность (5.25 ч.) и критическая важность услуги, особенно в зимний период, делают предсказание времени отключения отопления высокоценным для пользователя.

3) Приоритет №3: Электричество: Имеет наименьшую медианную длительность, но высокая значимость электроэнергии для современных бытовых и рабочих нужд требует включения этого типа в фичу предсказания.

Таким образом, фича предсказания времени отключения должна быть универсальной для всех типов, но акцент на точности и оперативности следует делать в первую очередь на отключениях горячей и холодной воды.

In [5]:
# ## 1.5. Анализ по инициатору (`initiator_name`)
# Выявление организаций, которые чаще всего являются инициаторами работ и, как следствие, отключений.

initiator_analysis = blackouts_df.groupby('initiator_name').agg(
    count=('id', 'count'),
    total_duration_hours=('duration_hours', 'sum'),
    median_duration_hours=('duration_hours', 'median')
).sort_values(by='count', ascending=False).head(10)

print("--- ТОП-10 Инициаторов отключений ---")
print(initiator_analysis.round(2))

--- ТОП-10 Инициаторов отключений ---
                                                 count  total_duration_hours  median_duration_hours
initiator_name                                                                                     
МУПВ ВПЭС (электрические сети)                    2431              13134.85                   4.12
КГУП «Приморский водоканал»                       1415              13918.03                   7.13
ООО "Мингородок"                                   875               5716.98                   3.72
ООО "Невельского"                                  777               7795.53                   4.10
МУПВ ВПЭС (тепловые сети Первомайского района)     751              98234.93                  14.27
МУПВ ВПЭС (тепловые сети)                          699             122782.58                  24.02
ООО "УК "Комплекс коммунальных услуг"              624              11153.87                   6.57
ООО "Управляющая компания Первомайского района"    592        

Вывод по Анализу Инициаторов Отключений
Анализ ТОП-10 инициаторов отключений показывает, какие организации генерируют наибольшее количество проблем и определяют самые продолжительные простои, что является важным контекстом для обоснования фичи предсказания.

Ключевые наблюдения:

Лидеры по частоте:

Наибольшее количество отключений приходится на МУПВ ВПЭС (электрические сети) (2431 случай) и КГУП «Приморский водоканал» (1415 случаев). Это логично, поскольку они отвечают за наиболее частые типы отключений (электричество и вода), как показано в Выводе 1.4.

Лидеры по продолжительности (по медиане):

МУПВ ВПЭС (тепловые сети) (медиана 24.02 ч.) и МУПВ ВПЭС (тепловые сети Первомайского района) (медиана 14.27 ч.) ответственны за отключения с самой высокой медианной длительностью.

Эти данные указывают, что хотя отключения тепла и горячей воды (за которые они в основном отвечают) происходят реже, чем отключения электроэнергии, их устранение занимает значительно больше времени.

Управляющие компании (УК):

В списке также присутствует значительное количество управляющих компаний (например, ООО "Мингородок", ООО "Невельского", ООО "УК "Комплекс коммунальных услуг"), что говорит о том, что существенная доля проблем связана не только с магистральными сетями, но и с внутридомовыми авариями или работами, которые проводятся самими УК.

Значение для фичи предсказания:
Выявление инициаторов позволяет понять, что проблема имеет две основные природы:

1) Массовые и относительно короткие проблемы (Электричество/Вода от крупных сетевых организаций).

2) Редкие, но крайне продолжительные проблемы (Отопление/Горячая вода от тепловых сетей).

Вывод: Поскольку отключения, инициированные тепловыми сетями ВПЭС, длятся многократно дольше (медиана до 24 часов) по сравнению с другими (медиана 3-7 часов), предсказание времени восстановления для этих событий имеет наивысшую пользовательскую ценность. Необходимость провести сутки без отопления или горячей воды требует максимально точного и своевременного информирования.

### Сводный Итог Этапа 1
Анализ таблицы blackouts подтвердил, что фича предсказания времени отключения является значимым улучшением:

Масштаб: Более 25 тысяч отключений и почти 800 тысяч совокупных часов простоя демонстрируют высокий риск для пользователей.

Актуальность: Типичное отключение длится около 5 часов (медиана), что достаточно долго для создания серьезных неудобств.

Критичность: Проблемы с теплом и водой (особенно инициированные тепловыми сетями МУПВ ВПЭС) являются самыми продолжительными (медиана до 24 часов), что делает предсказание для них  важным для планирования жизни и минимизации ущерба

### Этап 2: Географическое распределение проблемы (Пострадавшие здания и районы)

 На этом этапе мы свяжем данные об отключениях (`blackouts`) с географической информацией (`buildings`, `districts`, `folk_districts`), чтобы показать, что проблема распределена **неравномерно** и сконцентрирована в определенных зонах. Это дополнительно усиливает обоснование ценности фичи предсказания для жителей этих "горячих точек".

 **Цель:**
1.  Оценить общее число зданий, затронутых отключениями.
2.  Выявить самые проблемные **Официальные районы** (`districts`) по частоте и длительности отключений.
3.  Выявить самые проблемные **Неофициальные районы** (`folk_districts`) для релевантности пользовательскому опыту.

In [6]:
#  2.1. Загрузка и подготовка данных

# Загружаем необходимые таблицы (`buildings`, `blackouts_buildings`, `districts`, `folk_districts`) и выполняем первичное объединение.

import sqlite3
import pandas as pd
import numpy as np

# Установка опций отображения
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Предполагается, что conn уже установлен из Этапа 1
DB_PATH = '../databases/dataset.db' 
conn = sqlite3.connect(DB_PATH)

# Загрузка дополнительных таблиц
query_bb = "SELECT * FROM blackouts_buildings;"
blackouts_buildings_df = pd.read_sql_query(query_bb, conn)

query_buildings = "SELECT id, district_id, folk_district_id, big_folk_district_id, is_fake FROM buildings;"
buildings_df = pd.read_sql_query(query_buildings, conn)

query_districts = "SELECT id, name FROM districts;"
districts_df = pd.read_sql_query(query_districts, conn)

query_folk_districts = "SELECT id, name FROM folk_districts;"
folk_districts_df = pd.read_sql_query(query_folk_districts, conn)

# Используем blackouts_df с duration_hours из Этапа 1
# Если blackouts_df не сохранен, его нужно загрузить и обработать снова:
if 'blackouts_df' not in locals():
    blackouts_df = pd.read_sql_query("SELECT id, start_date, end_date, type FROM blackouts;", conn)
    blackouts_df['start_date'] = pd.to_datetime(blackouts_df['start_date'])
    blackouts_df['end_date'] = pd.to_datetime(blackouts_df['end_date'])
    blackouts_df['duration'] = blackouts_df['end_date'] - blackouts_df['start_date']
    blackouts_df['duration_hours'] = blackouts_df['duration'].dt.total_seconds() / 3600
    blackouts_df = blackouts_df[blackouts_df['duration_hours'] > 0] # Очистка

print(f"Всего отключений: {len(blackouts_df)}")
print(f"Связей отключение-здание: {len(blackouts_buildings_df)}")
print(f"Всего зданий: {len(buildings_df)}")

Всего отключений: 25264
Связей отключение-здание: 175590
Всего зданий: 58199


In [7]:
# ## 2.2. Оценка охвата зданий

# Выясним, какое количество уникальных зданий было затронуто отключениями, чтобы оценить общий охват проблемы.

# Уникальные здания, затронутые хотя бы одним отключением
affected_buildings_count = blackouts_buildings_df['building_id'].nunique()
total_buildings_count = buildings_df['id'].nunique()

print("--- Оценка охвата зданий ---")
print(f"Общее количество уникальных зданий в базе: {total_buildings_count:,}")
print(f"Количество зданий, затронутых отключениями: {affected_buildings_count:,}")
print(f"Процент затронутых зданий: {(affected_buildings_count / total_buildings_count * 100):.2f}%")

--- Оценка охвата зданий ---
Общее количество уникальных зданий в базе: 58,199
Количество зданий, затронутых отключениями: 21,159
Процент затронутых зданий: 36.36%


In [11]:
# ## 2.3. Анализ по Официальным Районам (`districts`)
# 
# Объединяем данные, чтобы получить метрики отключений по официальным административным районам.
# 
# **Схема объединения:** # `blackouts` $\rightarrow$ `blackouts_buildings` $\rightarrow$ `buildings` $\rightarrow$ `districts`

# %%
# 1. Объединение отключений и зданий
blackouts_with_buildings = blackouts_buildings_df.merge(
    blackouts_df[['id', 'duration_hours']],
    left_on='blackout_id',
    right_on='id',
    how='inner'
).rename(columns={'id': 'blackout_id_clean'}) # Переименовываем id отключения, чтобы не путать

# 2. Добавление информации о районах к отключениям
blackouts_with_geo = blackouts_with_buildings.merge(
    buildings_df[['id', 'district_id']],
    left_on='building_id',
    right_on='id',
    how='inner'
).rename(columns={'id': 'building_id_clean'})

# 3. Группировка и агрегация по Официальному району
district_analysis = blackouts_with_geo.groupby('district_id').agg(
    total_blackouts=('blackout_id', 'nunique'), # Считаем уникальные отключения
    total_duration_hours=('duration_hours', 'sum'),
    median_duration_hours=('duration_hours', 'median')
)

district_analysis = district_analysis.reset_index().merge( # Сброс индекса, чтобы district_id стал столбцом
    districts_df.rename(columns={'id': 'district_id', 'name': 'district_name'}),
    on='district_id',
    how='left'
).set_index('district_name').sort_values(by='total_blackouts', ascending=False)


# Дополнительно: Количество уникальных зданий в каждом районе (для нормализации)
buildings_per_district = buildings_df.groupby('district_id')['id'].nunique().to_frame('total_buildings')
# ИСПРАВЛЕНИЕ: Сбрасываем индекс district_id, чтобы он стал столбцом для слияния
buildings_per_district = buildings_per_district.reset_index() 

# Теперь buildings_per_district содержит столбец 'district_id'.
# districts_df содержит 'id' и 'name'.
# Слияние для получения имени района:
buildings_per_district = buildings_per_district.merge(
    districts_df, 
    left_on='district_id', # Используем district_id из buildings_per_district
    right_on='id',         # Используем id из districts_df
    how='left'
).set_index('name')['total_buildings'] # Устанавливаем name как индекс и выбираем нужный столбец

# Финальное слияние: используем left_index=True для слияния по индексу (названию района)
district_analysis = district_analysis.merge(
    buildings_per_district, 
    left_index=True, 
    right_index=True, 
    how='left'
)

# Нормализация: частота отключений на одно здание
district_analysis['blackouts_per_building'] = district_analysis['total_blackouts'] / district_analysis['total_buildings']

print("--- Анализ отключений по Официальным районам ---")
print(district_analysis[['total_blackouts', 'blackouts_per_building', 'median_duration_hours', 'total_duration_hours']].head(10).round(2))

--- Анализ отключений по Официальным районам ---
                      total_blackouts  blackouts_per_building  median_duration_hours  total_duration_hours
district_name                                                                                             
Ленинский район                  7684                    1.86                   8.68            2675451.77
Первомайский район               6518                    1.75                   8.00            1927361.70
Первореченский район             5778                    1.47                   8.00            1999635.88
Советский район                  3214                    0.17                   8.00            1963813.60
Фрунзенский район                2389                    0.70                   8.10            1558756.43


Вывод по Анализу Отключений в Официальных Районах
Анализ географического распределения отключений по официальным районам подтверждает, что проблема является локализованной и острой для определенных территорий, что критически важно для определения приоритетов при внедрении фичи предсказания.
Ключевые наблюдения:

Лидеры по частоте и интенсивности (Горячие точки):

1) Ленинский район лидирует по абсолютному числу отключений (7684) и имеет самую высокую частоту отключений на одно здание (1.86). Это означает, что в среднем каждое здание в районе было затронуто отключениями почти дважды.
2) Первомайский район (6518 отключений, 1.75 на здание) и Первореченский район (5778 отключений, 1.47 на здание) также являются зонами с критически высокой частотой.

Продолжительность отключений:

Все ведущие районы (Ленинский, Первомайский, Первореченский) демонстрируют высокую медианную длительность отключения — около 8 часов (от 8.00 до 8.68 часов). Это более чем в полтора раза выше, чем средняя медиана для всего города (4.68 часа, согласно Этапу 1). Это указывает на то, что проблемы в этих районах не только часты, но и требуют длительного времени на устранение.

Аномалия Советского района:

Советский район имеет относительно высокое общее время простоя (total_duration_hours) и медианную длительность (8.00 ч.), но при этом — аномально низкую частоту отключений на здание (0.17). Это может указывать на:
1) наличие небольшого числа чрезвычайно продолжительных, масштабных аварий, затронувших множество зданий одновременно;
2) возможно, большое количество "фейковых" зданий в базе, которые завышают знаменатель (total_buildings). Дополнительный анализ необходим, но в целом проблема здесь качественно иная, чем в Ленинском/Первомайском.

Обоснование приоритетов для фичи предсказания:

Высокая частота (blackouts_per_building > 1.4) в сочетании с высокой длительностью (медиана $\approx 8$ часов) в Ленинском, Первомайском и Первореченском районах создает наиболее острую потребность в фиче предсказания.Для жителей этих районов фича будет иметь максимальную ценность, так как она поможет планировать свою жизнь в условиях регулярных и долгих коммунальных проблем.

In [13]:
# ## 2.4. Анализ по Народным Районам (`folk_districts`)

# Повторяем анализ, используя народные районы. Эти данные более точно отражают восприятие местоположения пользователями.

# **Схема объединения:** # `blackouts_with_buildings` $\rightarrow$ `buildings` (по `folk_district_id`) $\rightarrow$ `folk_districts`

# 1. Объединение с Народным районом
blackouts_with_folk_geo = blackouts_with_buildings.merge(
    buildings_df[['id', 'folk_district_id']],
    left_on='building_id',
    right_on='id',
    how='inner'
)

# 2. Группировка и агрегация по Народному району
folk_district_analysis = blackouts_with_folk_geo.groupby('folk_district_id').agg(
    total_blackouts=('blackout_id', 'nunique'),
    median_duration_hours=('duration_hours', 'median')
)

folk_district_analysis = folk_district_analysis.reset_index().merge( # Сброс индекса
    folk_districts_df.rename(columns={'id': 'folk_district_id', 'name': 'folk_district_name'}),
    on='folk_district_id',
    how='left'
).set_index('folk_district_name').sort_values(by='total_blackouts', ascending=False)

# Дополнительно: Количество уникальных зданий в Народном районе
buildings_per_folk_district = buildings_df.groupby('folk_district_id')['id'].nunique().to_frame('total_buildings')
# ИСПРАВЛЕНИЕ: Сбрасываем индекс, чтобы он стал столбцом для слияния
buildings_per_folk_district = buildings_per_folk_district.reset_index() 

# Теперь buildings_per_folk_district содержит столбец 'folk_district_id'.
# folk_districts_df содержит 'id' и 'name'.
# Слияние для получения имени района:
buildings_per_folk_district = buildings_per_folk_district.merge(
    folk_districts_df, 
    left_on='folk_district_id', 
    right_on='id', 
    how='left'
).set_index('name')['total_buildings'] # Устанавливаем name как индекс

# Финальное слияние: используем left_index=True для слияния по индексу (названию района)
folk_district_analysis = folk_district_analysis.merge(
    buildings_per_folk_district, 
    left_index=True, 
    right_index=True, 
    how='left'
)

# Нормализация
folk_district_analysis['blackouts_per_building'] = folk_district_analysis['total_blackouts'] / folk_district_analysis['total_buildings']

print("--- ТОП-10 Народных районов по частоте отключений ---")
print(folk_district_analysis[['total_blackouts', 'blackouts_per_building', 'median_duration_hours']].head(10).round(2))

--- ТОП-10 Народных районов по частоте отключений ---
                      total_blackouts  blackouts_per_building  median_duration_hours
folk_district_name                                                                  
64, 71 микрорайон                2814                    4.92                   8.00
Чуркин                           2153                    2.44                   8.00
Тихая                            1523                    4.85                   9.37
Центр                            1514                    1.54                   8.17
БАМ                              1487                    2.52                   8.62
Столетие                         1319                    3.07                   8.00
Окатовая                         1303                    5.19                   8.00
Гризодубова-Сафонова             1218                    3.77                   8.07
Баляева                          1201                    5.29                   8.73
3 рабочая  

Ключевые наблюдения по Народным районам:

Лидеры по интенсивности (Частота на здание):

Районы Баляева (5.29 отключений на здание), Окатовая (5.19), и 64, 71 микрорайон (4.92) имеют критически высокую частоту отключений. Это означает, что в этих районах каждое здание сталкивается с отключениями 5 раз за анализируемый период.

Лидеры по абсолютному числу:

64, 71 микрорайон (2814 отключений) и Чуркин (2153 отключения) лидируют по общему количеству событий, что указывает на высокую концентрацию проблем.

Длительность отключений:Практически все районы из ТОП-10 имеют высокую медианную длительность отключений, колеблющуюся от 8.00 до 9.37 часов.

## Этап 3: Окончательное обоснование и резюме

In [16]:
## 3.1. Сводная таблица ключевых метрик

# Сводим наиболее критичные показатели, доказывающие масштаб и остроту проблемы.

# В реальном Notebook здесь были бы переменные, содержащие результаты предыдущих этапов.
# Для целей резюме, мы используем полученные ранее значения.

# Метрики Этапа 1 (Масштаб проблемы)
total_blackouts = 25264
median_duration_city = 4.68
total_duration_hours = 789091.80

# Метрики Этапа 1 (Тип услуг)
water_share = 73.03  # cold_water + hot_water
max_median_duration_by_type = 6.00 # hot_water

# Метрики Этапа 2 (География)
affected_buildings_share = 35.0 # (Предположим, что 35% зданий затронуто)
max_blackouts_per_building = 5.29 # Баляева
max_median_duration_by_district = 9.37 # Тихая

# Создание сводной таблицы для презентации
summary_data = {
    'Метрика': [
        'Общее количество отключений',
        'Совокупное время простоя (часы)',
        'Медианная длительность по городу (часы)',
        'Доля отключений Вода/Отопление',
        'Макс. медианная длительность (тип)',
        'Макс. частота на здание (в районе)',
        'Макс. медианная длительность (в районе)'
    ],
    'Значение': [
        f"{total_blackouts:,}",
        f"{total_duration_hours:,.0f}",
        f"{median_duration_city:.2f}",
        f"{water_share:.1f}%",
        f"{max_median_duration_by_type:.2f} ч.",
        f"{max_blackouts_per_building:.2f} раз",
        f"{max_median_duration_by_district:.2f} ч."
    ],
    'Единица измерения': ['событий', 'часов', 'часов', '%', 'часы', 'раз', 'часы']
}

summary_df = pd.DataFrame(summary_data)
print("--- Сводные метрики проблемы отключений ---")
print(summary_df)

--- Сводные метрики проблемы отключений ---
                                   Метрика  Значение Единица измерения
0              Общее количество отключений    25,264           событий
1          Совокупное время простоя (часы)   789,092             часов
2  Медианная длительность по городу (часы)      4.68             часов
3           Доля отключений Вода/Отопление     73.0%                 %
4       Макс. медианная длительность (тип)   6.00 ч.              часы
5       Макс. частота на здание (в районе)  5.29 раз               раз
6  Макс. медианная длительность (в районе)   9.37 ч.              часы


Проблема отключений в городе является регулярной (более 25 тысяч событий), продолжительной (медиана 4.68 ч.) и концентрированной (до 5.29 раз на здание). Эти факторы создают высокий уровень стресса и неопределенности для жителей, что делает фичу предсказания времени восстановления критически необходимой.

Предлагаемая фича решает эту проблему, трансформируя пассивное ожидание в управляемый процесс, что напрямую повышает лояльность пользователей и снижает нагрузку на диспетчерские службы.